## TODO

1. Validations of relations between different cells. Like "Diabetes" show -> Member has Diab Type 2 or not. And "Diab_Type" shows type. So whereever Diabetes == 1, Diab_Type should be == "Type 2"!!

2. Visualisations and Business related answers!! (Ishika, can you please look aggresively into this?)
3. Figure out using - "con_visit...." , "pot_visit...." ,"rx_ther...." (Wide format variables!!) 
4. Separate out different categories of diseases - respiratory, etc etc

In [104]:
import pandas as pd
import matplotlib as plot
import numpy as np
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
import re

In [105]:
#import data from csv
base_path = "/Users/apple/Documents/gatech/humana_analytics/humana_2018/" # path to humana folder

data = pd.read_csv(base_path + "/data/TAMU_FINAL_DATASET_2018.csv")

In [106]:
data.head()
data.describe() 
data.shape
#contains one lakh rows

(100000, 448)

Take Aways -

1. One Lakh rows
2. Check for NAs, because there are - Impute them!
3. Age distribution of the population is on the older side - min age is 40, max is 95

In [107]:
#Cleaning the dataset a bit to make it more general

#standardise column names
data.columns = [s.lower() for s in data.columns]

# find columns with na values
def showmissing(df_train):
    return df_train.columns[df_train.isnull().any()].tolist()

#missing data percentages
(data[showmissing(data)].isnull().sum().sort_values(ascending = False)/100000)*100


diab_type                        64.693
decile_struggle_med_lang         25.215
population_density_centile_us    12.179
online_purchaser                 12.179
dwelling_type                    12.179
education_level                  12.179
length_residence                 12.179
population_density_centile_st    12.179
num_person_household             12.179
college                          12.179
est_bmi_decile                   12.179
online_user                      12.179
pct_above_poverty_line           12.179
pct_below_poverty_line           12.179
home_value                       12.179
est_net_worth                    12.179
est_income                       12.179
index_health_ins_engage          12.179
index_health_ins_influence       12.179
pcp_assignment                    0.157
mco_hlvl_plan_cd                  0.033
mco_prod_type_cd                  0.033
hospice_ind                       0.032
esrd_ind                          0.032
lis                               0.030


In [108]:
'''


data['recon_rx_risk_score_nbr'].describe() 
data['recon_ma_risk_score_nbr'].describe()
data['population_density_centile_st']

'''

"\n\n\ndata['recon_rx_risk_score_nbr'].describe() \ndata['recon_ma_risk_score_nbr'].describe()\ndata['population_density_centile_st']\n\n"

In [109]:
#more exploration with respect to target variable

# We can see that ami_flag is relatively varied among Males and Females. 
pd.crosstab(index=data['sex_cd'], columns=data['ami_flag'], margins=True)
 

ami_flag,0,1,All
sex_cd,,,
F,55476,1292,56768
M,41771,1431,43202
All,97247,2723,99970


In [110]:
### Cleaning sex_cd Column

# Cleaning sex_cd - there are 0.03% NAs in this column. Lets see how we can impute it - or we can remove those rows.
data.sex_cd.unique() #there are no "U" as expected according to data dictionary. Assuming NAs to be Us
data.sex_cd.isnull().sum()  # 30 values. 

#Interestingly all sex_cd = NAs are people with age=40
data[data["sex_cd"].isnull()]['age']

buffer = data[data["sex_cd"].isnull()]

#We can see that we cannot totally remove the rows believing it to be a data problem because only 10 of the columns are totally NA. 
(buffer[showmissing(buffer)].isnull().sum().sort_values(ascending = False)/30)*100

# For now lets change all the sex_cd = NA with "U"

data["sex_cd"].fillna("U", inplace = True)
data.sex_cd.unique() # F- Female, M- Male, U - Unknown

# encoding it
sex_dummies = pd.get_dummies(data["sex_cd"])
data = data.join(sex_dummies)

In [111]:
## Handling all the NAs value in the dataset. 

#Step1: Understand why the data has NA. Can it mean something while keeping the domain in mind
#Step2: Action to take - remove the rows or impute?
#Step3: Return a dataset with no NAs

col_with_nas = showmissing(data)

# mco_hlvl_plan_cd and mco_hlvl_type_cd
data.mco_hlvl_plan_cd.unique()  # ['MAPD', 'MA', nan]
data.mco_prod_type_cd.unique()  # ['LPPO', 'PFFS', 'HMO', 'RPPO', nan]

#both are NAs for the same person
data[data["mco_hlvl_plan_cd"].isnull()]['id'] == data[data["mco_prod_type_cd"].isnull()]['id'] 

# Decision - Both of these variables are intutively less likely to determine whether a patient will get AMI. 
# So let the NAs be. 

data[data["est_income"].isnull()]
data.pcp_assignment.unique()

#diab_type : Hypothesis - this is NULL whenever diabetes == 0 
sum(data[data['diab_type'].isnull()]['id'] == data[data['diabetes'] == 0]['id']) == sum(data['diabetes'] == 0)
data["diab_type"].fillna("NO", inplace = True)

#esrd_ind 


In [112]:
trial_cols = ["id","sex_cd","age","population_density_centile_us" , "online_purchaser", "dwelling_type","education_level","length_residence","population_density_centile_st","num_person_household","college","est_bmi_decile","online_user","pct_above_poverty_line","pct_below_poverty_line","home_value","est_net_worth","est_income","index_health_ins_engage", "index_health_ins_influence"]   

#df_col = data[trial_cols]
#df_col[df_col["population_density_centile_st"].isnull()] 

#sorted(col_with_nas)
# Remarks - All the NAs for these columns occur for the same people. This has to be a data collection error! 
# 12179 rows

In [113]:
trial_cols1 = ["lis","institutional","dual","orig_reas_entitle_cd","id","age","sex_cd"]
df_col1 = data[trial_cols1]
df_col1[df_col1["lis"].isnull()]

#These are the same set of people who had their "sex_cd" as NA. Data source must be the same as sex_cd!

,lis,institutional,dual,orig_reas_entitle_cd,id,age,sex_cd
5040,NaN,NaN,NaN,NaN,5041,40,U
6049,NaN,NaN,NaN,NaN,6050,40,U
6573,NaN,NaN,NaN,NaN,6574,40,U
15596,NaN,NaN,NaN,NaN,15597,40,U
16040,NaN,NaN,NaN,NaN,16041,40,U
16157,NaN,NaN,NaN,NaN,16158,40,U
17056,NaN,NaN,NaN,NaN,17057,40,U
18352,NaN,NaN,NaN,NaN,18353,40,U
23929,NaN,NaN,NaN,NaN,23930,40,U
32216,NaN,NaN,NaN,NaN,32217,40,U


In [114]:
#trial_cols2 = ["hospice_ind","esrd_ind","id","age","sex_cd"]
trial_cols2 = ["mco_hlvl_plan_cd","mco_prod_type_cd","id","age","sex_cd"]

df_col2 = data[trial_cols2]
df_col2[df_col2["mco_hlvl_plan_cd"].isnull()]

# For ["hospice_ind","esrd_ind"] - Same people like above, except two Males - id = [68995,99088]
# For ["mco_hlvl_plan_cd","mco_prod_type_cd"] - Same like above except three Males - id [19348,55303,78200]

,mco_hlvl_plan_cd,mco_prod_type_cd,id,age,sex_cd
5040,NaN,NaN,5041,40,U
6049,NaN,NaN,6050,40,U
6573,NaN,NaN,6574,40,U
15596,NaN,NaN,15597,40,U
16040,NaN,NaN,16041,40,U
16157,NaN,NaN,16158,40,U
17056,NaN,NaN,17057,40,U
18352,NaN,NaN,18353,40,U
19347,NaN,NaN,19348,84,M
23929,NaN,NaN,23930,40,U


In [115]:
# Final NAs analysis summary

#ids_30 - ids of people who have their sex_cd = U and ["lis","institutional","dual","orig_reas_entitle_cd"] = NA
ids_30 = list(data[data.sex_cd == "U"]["id"] )

#ids_2 - addition to ids_30, two more person who have ["hospice_ind","esrd_ind"] = NA
ids_2 = list(data[(data.sex_cd != "U") & (data["hospice_ind"].isnull())]["id"])

#ids_3 - addition to ids_30, three more person who have ["mco_hlvl_plan_cd","mco_prod_type_cd"] = NA
ids_3 = list(data[(data.sex_cd != "U") & (data["mco_hlvl_plan_cd"].isnull())]["id"])

#pcp_assignment - Waiting to hear back from Humana Team on the meaning behind this

# All Variables with 12.79% NAs are demographic variables! Ignore them for now
#["population_density_centile_us" , "online_purchaser", "dwelling_type","education_level","length_residence","population_density_centile_st","num_person_household","college","est_bmi_decile","online_user","pct_above_poverty_line","pct_below_poverty_line","home_value","est_net_worth","est_income","index_health_ins_engage", "index_health_ins_influence"]   



In [116]:
# filtering out variables of interest 

#personal details
numeric_col = ["age"]
primary_cols = ["M","F"]
#demographic cols with NAs
demographic_cols = ["population_density_centile_us" , "online_purchaser", "dwelling_type","education_level","length_residence","population_density_centile_st","num_person_household","college","est_bmi_decile","online_user","pct_above_poverty_line","pct_below_poverty_line","home_value","est_net_worth","est_income","index_health_ins_engage", "index_health_ins_influence"]

#health variables
adherent_measure = ["ace_elig","ace_pass","amm","amm_gap","diab_pass","diab_elig","statin_elig","statin_pass","bcs","bcs_gap","col","col_gap"]

disease_1 = ["ckd","esrd","hyperlipid","hypertension","renal"]
disease_cv = ["cv_cad","cv_cer","cv_chf","cv_cir","cv_hdz","cv_pvd","cv_sns"]
disease_resp = ["res_ast","res_alg","res_copd","res_fail","res_inf"]
disease_movement = ["arth","muscul_bn","muscul_oth","osteo"]
disease_sugar = ["prediabetes","diab_complications","diabetes","cdc","cdc_eye_gap" ,
                 "cdc_hbapoor_gap","cdc_hbatest_gap","cdc_nph_gap"]

scores = ["recon_ma_risk_score_nbr","recon_rx_risk_score_nbr"]







In [117]:
# Removing 32 persons whose some imp health variables are NAs
# For now removing it, we can think of how to improve this later

remove_ids = ids_30 + ids_2 + ids_3
data = data[~data["id"].isin(remove_ids)]
data.shape

(99965, 451)

In [118]:
#Split into train and test

data.ami_flag = pd.Categorical(data.ami_flag)

# converting categorical into categorical
data[primary_cols + adherent_measure + disease_1 +disease_cv +disease_resp+disease_movement+disease_sugar] = data[primary_cols + adherent_measure + disease_1 +disease_cv +disease_resp+disease_movement+disease_sugar].apply( pd.Categorical,axis=0)




target_df = data['ami_flag']

#choosing the diseases variables

var = numeric_col +primary_cols + adherent_measure + disease_1 +disease_cv +disease_resp+disease_movement+disease_sugar+scores
#variable_df = data.drop('ami_flag', axis = 1)
variable_df = data[var]

X_train, X_test, y_train, y_test = train_test_split(variable_df, target_df, test_size=0.2) #20% data in test

In [119]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [120]:
# Trying out linear regression

linear_reg = linear_model.LogisticRegression()

linear_reg.fit(X_train,y_train)

predictions = linear_reg.predict(X_test)

print('Coefficients: \n', linear_reg.coef_)

Coefficients: 
 [[ 0.01859525 -1.77425978 -2.01738221  0.08731873 -0.15828689 -0.08067926
   0.13499317 -0.18356453  0.09067573 -0.06339704 -0.26031201 -0.03209364
   0.06540463 -0.17517517  0.13911025  0.08652976  0.67474032 -0.09076018
   0.09222126  0.0036786   0.90577763  0.15670069  0.33271298  0.10997715
   0.1124835   0.0893864   0.39463088 -0.08314075 -0.10838409  0.20993323
   0.04596512  0.0187266  -0.0757326   0.0884803  -0.15804185  0.10338117
  -0.18725811  0.32564041  0.0740192  -0.02541988  0.11826334  0.17287171
   0.15079186 -0.18255016  0.06769193 -0.006852  ]]


In [121]:
pd.crosstab(index=y_test, columns=predictions)

linear_reg.predict_proba(X_test)

array([[0.98168587, 0.01831413],
       [0.96381886, 0.03618114],
       [0.94453476, 0.05546524],
       ...,
       [0.93439374, 0.06560626],
       [0.90504652, 0.09495348],
       [0.94451013, 0.05548987]])

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [123]:
clf = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [124]:
print(clf.feature_importances_)

[0.04282215 0.00633274 0.00679991 0.00265534 0.00311595 0.00198144
 0.00134299 0.00166798 0.00292584 0.00429426 0.00675828 0.00363695
 0.00170176 0.00478216 0.00232042 0.01404302 0.02749226 0.00563501
 0.00327871 0.01226707 0.24618413 0.02461066 0.103638   0.02211397
 0.07313471 0.01020655 0.08798199 0.00252981 0.00132976 0.0099847
 0.02245827 0.0032727  0.00163539 0.00427831 0.00204136 0.00183508
 0.00195933 0.03608956 0.01466062 0.00359708 0.0042164  0.00500132
 0.00372675 0.00221559 0.10210377 0.05333997]


In [125]:

pd.crosstab(index=y_test, columns=predictions)

clf.predict_proba(X_test)

array([[0.98017662, 0.01982338],
       [0.96668881, 0.03331119],
       [0.94146058, 0.05853942],
       ...,
       [0.95375496, 0.04624504],
       [0.95084543, 0.04915457],
       [0.94595756, 0.05404244]])

In [ ]:
from sklearn import svm

clf1 = svm.SVC()
clf1.fit(X_train,y_train)


In [128]:
y_train.dtypes

CategoricalDtype(categories=[0, 1], ordered=False)